<a href="https://colab.research.google.com/github/vieri03/Prediction-of-Apple-Ripeness/blob/main/GLCM4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from skimage import io
from scipy import stats
from skimage.feature import greycoprops
import glob
import csv

def offset(length, angle):
    """Return the offset in pixels for a given length and angle"""
    dv = length * np.sign(-np.sin(angle)).astype(np.int32)
    dh = length * np.sign(np.cos(angle)).astype(np.int32)
    return dv, dh

def crop(img, center, win):
    """Return a square crop of img centered at center (side = 2*win + 1)"""
    row, col = center
    side = 2*win + 1
    first_row = row - win
    first_col = col - win
    last_row = first_row + side    
    last_col = first_col + side
    return img[first_row: last_row, first_col: last_col]

def cooc_maps(img, center, win, d=[1], theta=[0], levels=256):
    """
    Return a set of co-occurrence maps for different d and theta in a square 
    crop centered at center (side = 2*w + 1)
    """
    shape = (2*win + 1, 2*win + 1, len(d), len(theta))
    cooc = np.zeros(shape=shape, dtype=np.int32)
    row, col = center
    Ii = crop(img, (row, col), win)
    for d_index, length in enumerate(d):
        for a_index, angle in enumerate(theta):
            dv, dh = offset(length, angle)
            Ij = crop(img, center=(row + dv, col + dh), win=win)
            cooc[:, :, d_index, a_index] = encode_cooccurrence(Ii, Ij, levels)
    return cooc

def encode_cooccurrence(x, y, levels=256):
    """Return the code corresponding to co-occurrence of intensities x and y"""
    return x*levels + y

def decode_cooccurrence(code, levels=256):
    """Return the intensities x, y corresponding to code"""
    return code//levels, np.mod(code, levels)    

def compute_glcms(cooccurrence_maps, levels=256):
    """Compute the cooccurrence frequencies of the cooccurrence maps"""
    Nr, Na = cooccurrence_maps.shape[2:]
    glcms = np.zeros(shape=(levels, levels, Nr, Na), dtype=np.float64)
    for r in range(Nr):
        for a in range(Na):
            table = stats.itemfreq(cooccurrence_maps[:, :, r, a])
            codes = table[:, 0]
            freqs = table[:, 1]/float(table[:, 1].sum())
            i, j = decode_cooccurrence(codes, levels=levels)
            glcms[i, j, r, a] = freqs
    return glcms

def compute_props(glcms, props=('contrast',)):
    """Return a feature vector corresponding to a set of GLCM"""
    Nr, Na = glcms.shape[2:]
    features = np.zeros(shape=(Nr, Na, len(props)))
    for index, prop_name in enumerate(props):
        features[:, :, index] = greycoprops(glcms, prop_name)
    return features.ravel()

def haralick_features(img, win, d, theta, levels, props):
    """Return a map of Haralick features (one feature vector per pixel)"""
    rows, cols = img.shape
    margin = win + max(d)
    arr = np.pad(img, margin, mode='reflect')
    n_features = len(d) * len(theta) * len(props)
    feature_map = np.zeros(shape=(rows, cols, n_features), dtype=np.float64)
    for m in range(rows):
        for n in range(cols):
            coocs = cooc_maps(arr, (m + margin, n + margin), win, d, theta, levels)
            glcms = compute_glcms(coocs, levels)
            feature_map[m, n, :] = compute_props(glcms, props)
    return feature_map

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/Colab Notebooks/

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


In [ ]:
import os
from os import listdir
from PIL import Image

path = '/content/gdrive/My Drive/Colab Notebooks/feature/'

for filename in listdir(path):
    if filename.endswith('.jpg'):
        try:
            img = Image.open(path + filename) # open the image file
            img.verify() # verify that it is, in fact an image
        except (IOError, SyntaxError) as e:
            print('Bad file:', filename)

In [ ]:
path = '/content/gdrive/My Drive/Colab Notebooks/feature/*.*'

filename = '/content/gdrive/My Drive/Colab Notebooks/result-image-twentyfifth.csv'

# open the file in the write mode
f = open(filename, 'w')

# create the csv writer
writer = csv.writer(f)
head = ["Number","Image", "contrast-1-0", "ASM-1-0", "energy-1-0", "homogeneity-1-0", "dissimilarity-1-0", 
        "contrast-1-45", "ASM-1-45", "energy-1-45", "homogeneity-1-45", "dissimilarity-1-45",
        "contrast-1-90", "ASM-1-90", "energy-1-90", "homogeneity-1-90", "dissimilarity-1-90",
        "contrast-1-135", "ASM-1-135", "energy-1-135", "homogeneity-1-135", "dissimilarity-1-135",
        "contrast-2-0", "ASM-2-0", "energy-2-0", "homogeneity-2-0", "dissimilarity-2-0", 
        "contrast-2-45", "ASM-2-45", "energy-2-45", "homogeneity-2-45", "dissimilarity-2-45",
        "contrast-2-90", "ASM-2-90", "energy-2-90", "homogeneity-2-90", "dissimilarity-2-90",
        "contrast-2-135", "ASM-2-135", "energy-2-135", "homogeneity-2-135", "dissimilarity-2-135",
        "contrast-3-0", "ASM-3-0", "energy-3-0", "homogeneity-3-0", "dissimilarity-3-0", 
        "contrast-3-45", "ASM-3-45", "energy-3-45", "homogeneity-3-45", "dissimilarity-3-45",
        "contrast-3-90", "ASM-3-90", "energy-3-90", "homogeneity-3-90", "dissimilarity-3-90",
        "contrast-3-135", "ASM-3-135", "energy-3-135", "homogeneity-3-135", "dissimilarity-3-135"]
writer.writerow(head)

image_no = 1
for file in sorted(glob.glob(path)): #sorted by name
  img = io.imread(file, as_gray=True)

  d = (1, 2, 3)
  theta = (0, np.pi/4, np.pi/2, 3*np.pi/4)
  props = ('contrast', 'ASM', 'energy', 'homogeneity', 'dissimilarity')
  levels = 8
  win = 19
  
  feature_map = haralick_features(img, win, d, theta, levels, props)
  feature_map[0, 0, :]

  #split distance
  array = np.array_split(feature_map[0, 0, :], 3)

  #split angles
  newarr_0 = np.array_split(array[0], 4)
  newarr_1 = np.array_split(array[1], 4)
  newarr_2 = np.array_split(array[2], 4)

  #split properties
  result_0_0 = np.array_split(newarr_0[0], 5)
  result_0_1 = np.array_split(newarr_0[1], 5)
  result_0_2 = np.array_split(newarr_0[2], 5)
  result_0_3 = np.array_split(newarr_0[3], 5)
  
  result_1_0 = np.array_split(newarr_1[0], 5)
  result_1_1 = np.array_split(newarr_1[1], 5)
  result_1_2 = np.array_split(newarr_1[2], 5)
  result_1_3 = np.array_split(newarr_0[3], 5)
  
  result_2_0 = np.array_split(newarr_2[0], 5)
  result_2_1 = np.array_split(newarr_2[1], 5)
  result_2_2 = np.array_split(newarr_2[2], 5)
  result_2_3 = np.array_split(newarr_0[3], 5)


  row = [image_no, file, result_0_0[0], result_0_0[1], result_0_0[2], result_0_0[3], result_0_0[4],
         result_0_1[0], result_0_1[1], result_0_1[2], result_0_1[3], result_0_1[4],
         result_0_2[0], result_0_2[1], result_0_2[2], result_0_2[3], result_0_2[4],
         result_0_3[0], result_0_3[1], result_0_3[2], result_0_3[3], result_0_3[4],
         result_1_0[0], result_1_0[1], result_1_0[2], result_1_0[3], result_1_0[4],
         result_1_1[0], result_1_1[1], result_1_1[2], result_1_1[3], result_1_1[4],
         result_1_2[0], result_1_2[1], result_1_2[2], result_1_2[3], result_1_2[4],
         result_1_3[0], result_1_3[1], result_1_3[2], result_1_3[3], result_1_3[4],
         result_2_0[0], result_2_0[1], result_2_0[2], result_2_0[3], result_2_0[4],
         result_2_1[0], result_2_1[1], result_2_1[2], result_2_1[3], result_2_1[4],
         result_2_2[0], result_2_2[1], result_2_2[2], result_2_2[3], result_2_2[4],
         result_2_3[0], result_2_3[1], result_2_3[2], result_2_3[3], result_2_3[4]]

  writer.writerow(row)
  image_no += 1

f.close()    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning:     `itemfreq` is deprecated!
    `itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
